#### Encode the ground truth latent space of Ag 

In [1]:
from pymatgen.core import Structure, Lattice
from pymatgen.io.ase import AseAtomsAdaptor
from chgnet.model import StructOptimizer

relaxer = StructOptimizer()

CHGNet initialized with 400,438 parameters
CHGNet will run on cpu


/mnt/c/Users/Lenovo/Downloads/chgnet/chgnet/graph/converter.py:70: UserWarning: `fast` algorithm is not available, using `legacy`
  warnings.warn(


In [3]:
# Reference structure of Ag from Materials Project, id=mp-124
ag_structure = Structure.from_file("/mnt/c/Users/Lenovo/Downloads/cdvae2/structures_GA/ag_cif/Ag_124.cif")
ag_atoms = AseAtomsAdaptor.get_atoms(ag_structure)

# Reference structure of Ag from Materials Project, id=mp-8566
ag_structure1 = Structure.from_file("/mnt/c/Users/Lenovo/Downloads/cdvae2/structures_GA/ag_cif/Ag_8566.cif")
ag_atoms1 = AseAtomsAdaptor.get_atoms(ag_structure1)

# Reference structure of Ag from Materials Project, id=mp-10597
ag_structure2 = Structure.from_file("/mnt/c/Users/Lenovo/Downloads/cdvae2/structures_GA/ag_cif/Ag_10597.cif")
ag_atoms2 = AseAtomsAdaptor.get_atoms(ag_structure2)

# Reference structure of Ag from Materials Project, id=mp-989737
ag_structure3 = Structure.from_file("/mnt/c/Users/Lenovo/Downloads/cdvae2/structures_GA/ag_cif/Ag_989737.cif")
ag_atoms3 = AseAtomsAdaptor.get_atoms(ag_structure3)

# Reference structure of Ag from Materials Project, id=mp-2646971
ag_structure4 = Structure.from_file("/mnt/c/Users/Lenovo/Downloads/cdvae2/structures_GA/ag_cif/Ag_2646971.cif")
ag_atoms4 = AseAtomsAdaptor.get_atoms(ag_structure4)

/home/fhj/anaconda3/lib/python3.10/site-packages/pymatgen/io/cif.py:1148: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


- Test if cosine similarity could distinguish 5 Ag structures encoded by CHGNet in latent space

In [33]:
b_ref = relaxer.calculator.model.graph_converter(ag_structure3)
h_ref = relaxer.calculator.model.forward([b_ref], return_crystal_feas=True)["crystal_fea"][0]

In [78]:
b_ref1 = relaxer.calculator.model.graph_converter(ag_structure4)
h_ref1 = relaxer.calculator.model.forward([b_ref1], return_crystal_feas=True)["crystal_fea"][0]

print(float(F.cosine_similarity(h_ref, h_ref1, dim=0, eps=1e-8)))

0.9994632005691528


- Verify the potential energy of Ag structures

In [42]:
result = relaxer.relax(atoms_to_structure(ag_atoms), fmax=1e-3, maxmove=0.2, verbose=False)
result["trajectory"].energies[-1]

-11.16677474975586

In [43]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.analysis.structure_matcher import StructureMatcher
from pymatgen.core import Structure

s = result['final_structure']
sga = SpacegroupAnalyzer(s, symprec=0.001)
ps = sga.get_conventional_standard_structure()

sm = StructureMatcher(scale=False, primitive_cell=False)
print(sm.fit(ag_structure, ps), sm.get_rms_dist(ag_structure, ps))

True (0.0, 0.0)


In [10]:
relax(ag_atoms4)
ag_atoms4.get_potential_energy()

/home/fhj/anaconda3/lib/python3.10/site-packages/ase/optimize/fire.py:65: VisibleDeprecationWarning: maxmove is deprecated; please use maxstep
  warnings.warn('maxmove is deprecated; please use maxstep',


-5.54036808013916

In [6]:
from ase.calculators.singlepoint import SinglePointCalculator
from ase.ga import set_raw_score
import io
from ase.constraints import ExpCellFilter
import contextlib
from ase.optimize import FIRE
import ase
from ase.build import niggli_reduce
from torch.nn import functional as F

def atoms_to_structure(atoms):
    lattice = Lattice(atoms.cell)
    symbols = atoms.get_chemical_symbols()
    positions = atoms.get_positions()
    return Structure(lattice, symbols, positions, coords_are_cartesian=True)

def finalize(atoms, energy=None, forces=None, stress=None):
    # Finalizes the atoms by attaching a SinglePointCalculator
    # and setting the raw score as the negative of the total energy
    atoms.wrap()
    calc = SinglePointCalculator(atoms, energy=energy, forces=forces,
                                 stress=stress)
    atoms.calc = calc
    raw_score = -atoms.get_potential_energy()
    set_raw_score(atoms, raw_score)

def relax(atoms, cellbounds=None):
    atoms.calc = relaxer.calculator  # assign model used to predict forces

    converged = False
    niter = 0
    stream = io.StringIO()
    with contextlib.redirect_stdout(stream):
        while not converged and niter < 10:
            if cellbounds is not None:
                cell = atoms.get_cell()
                if not cellbounds.is_within_bounds(cell):
                    niggli_reduce(atoms)
                cell = atoms.get_cell()
                if not cellbounds.is_within_bounds(cell):
                    # Niggli reduction did not bring the unit cell
                    # within the specified bounds; this candidate should
                    # be discarded so we set an absurdly high energy
                    finalize(atoms, 1e9)
                    return
                
            ecf = ExpCellFilter(atoms)
            dyn = FIRE(ecf, maxmove=0.2, logfile=None, trajectory=None)
            dyn.run(fmax=1e-3, steps=100)

            converged = dyn.converged()
            niter += 1
    
    dyn = FIRE(atoms, maxmove=0.2, logfile=None, trajectory=None)
    dyn.run(fmax=1e-2, steps=100)

    e = atoms.get_potential_energy()
    f = atoms.get_forces()
    s = atoms.get_stress()

    finalize(atoms, energy=e, forces=f, stress=s)